In [809]:
using Base.Iterators
using Roots
using IntervalSets
using JuMP
using Ipopt
using CompEcon
using QuantEcon
using BenchmarkTools
using LinearAlgebra
using Plots

┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1342
┌ Warning: Module JLLWrappers with build ID 1084643734684959 is missing from the cache.
│ This may mean JLLWrappers [692b3bcd-3c85-4b1f-b108-f13ce0eb3210] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1030
┌ Warning: Module JLLWrappers with build ID 1084643734684959 is missing from the cache.
│ This may mean JLLWrappers [692b3bcd-3c85-4b1f-b108-f13ce0eb3210] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1030
┌ Warning: Module JLLWrappers with build ID 1084643734684959 is missing from the cache.
│ This may mean JLLWrappers [692b3bcd-3c85-4b1f-b108-f13ce0eb3210] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1030
┌ Warning: Module JLLWrappers with build ID 1084643734684959 is missing from the cache.
│ This may mean JLLWrappers [692b3bcd-3c85-4b1f-b108-f13ce0eb3210] d

┌ Warning: Module JLLWrappers with build ID 1084643734684959 is missing from the cache.
│ This may mean JLLWrappers [692b3bcd-3c85-4b1f-b108-f13ce0eb3210] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1030
┌ Info: Skipping precompilation since __precompile__(false). Importing Libffi_jll [e9f186c6-92d2-5b65-8a66-fee21dc1b490].
└ @ Base loading.jl:1047
┌ Info: Precompiling Gettext_jll [78b55507-aeef-58d4-861c-77aaff3498b1]
└ @ Base loading.jl:1342
┌ Warning: Module JLLWrappers with build ID 1084643734684959 is missing from the cache.
│ This may mean JLLWrappers [692b3bcd-3c85-4b1f-b108-f13ce0eb3210] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1030
┌ Info: Skipping precompilation since __precompile__(false). Importing Gettext_jll [78b55507-aeef-58d4-861c-77aaff3498b1].
└ @ Base loading.jl:1047
┌ Info: Precompiling XML2_jll [02c8fc9c-b97f-50b9-bbe4-9be30ff0a78a]
└ @ Base loading.jl:1342
┌ Warning: M

┌ Warning: Module JLLWrappers with build ID 1084643734684959 is missing from the cache.
│ This may mean JLLWrappers [692b3bcd-3c85-4b1f-b108-f13ce0eb3210] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1030
┌ Info: Skipping precompilation since __precompile__(false). Importing x264_jll [1270edf5-f2f9-52d2-97e9-ab00b5d0237a].
└ @ Base loading.jl:1047
┌ Info: Precompiling x265_jll [dfaa095f-4041-5dcd-9319-2fabd8486b76]
└ @ Base loading.jl:1342
┌ Warning: Module JLLWrappers with build ID 1084643734684959 is missing from the cache.
│ This may mean JLLWrappers [692b3bcd-3c85-4b1f-b108-f13ce0eb3210] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1030
┌ Info: Skipping precompilation since __precompile__(false). Importing x265_jll [dfaa095f-4041-5dcd-9319-2fabd8486b76].
└ @ Base loading.jl:1047
┌ Info: Precompiling OpenSSL_jll [458c3c95-2e84-50aa-8efc-19380b2a3a95]
└ @ Base loading.jl:1342
┌ Warning: Module

### Model settings

In [788]:
# income shocks and their transition probabilities of the household 1
inc1 = [2/3, 4/3];
P1 = hcat(repeat([0.1 0.9], 2));

# income shocks and their transition probabilities of the household 2
inc2 = [2/3, 4/3];
P2 = hcat(repeat([0.1 0.9], 2));

# transition super-matrix of income shocks
R = kron(P2, P1);

# number of states
# (number of income states for HH 1 times
# number of income states for HH 2)
num_states = length(inc1) * length(inc2);

# Income matrix
# (col 1: HH 1's income, col 2: HH 2's income)
inc_mat = vec(collect(product(inc1, inc2)));

# Aggregate income in each state
inc_ag = sum.(inc_mat);


In [789]:
util = function(c, σ::Float64);
    if σ != 1.0 
        return (c^(1.0 - σ) - 1.0) / (1.0 - σ)
    elseif σ == 1.0
        return log(c)
    end
end;

util_prime = (c, σ::Float64) -> c^(- σ);

### Value of autarky

In [790]:
δ = 0.95 # time discount factor;
σ₁ = 1.0 # coefficient of relative risk aversion of HH1;
σ₂ = 1.0 # coefficient of relative risk aversion of HH2;
ϕ = 0.0 # punishment under autarky;

In [791]:
function Uaut_func(
        inc1,
        P1,
        inc2,
        P2,
        δ,
        σ₁,
        σ₂,
        ϕ,
        util, 
        util_prime
    )
    
    # Expected utility under autarky for HH1
    U1_aut = zeros(length(inc1));
    i = 1;
    diff = 1.0;
    while diff > 1e-12
        U1_aut_new = util.(inc1 .* (1.0 - ϕ), σ₁) .+ δ .* (P1 * U1_aut);
        diff = maximum(abs.(U1_aut_new - U1_aut));
        U1_aut = U1_aut_new;
        i += 1;
    end
    
    # Expected utility under autarky for HH2
    U2_aut = zeros(length(inc2));
    i = 1;
    diff = 1.0;
    while diff > 1e-12
        U2_aut_new = util.(inc2 .* (1.0 - ϕ), σ₁) .+ δ .* (P2 * U2_aut);
        diff = maximum(abs.(U2_aut_new - U2_aut));
        U2_aut = U2_aut_new;
        i += 1;
    end
    
    # Matrix of expected utilities of autarky
    Uaut = vec(collect(product(U1_aut, U2_aut)));
    return Uaut
end

Uaut = Uaut_func(inc1, P1, inc2, P2, δ, σ₁, σ₂, ϕ, util, util_prime);
Uaut_1 = [Uaut[i][1] for i in 1:num_states];
Uaut_2 = [Uaut[i][2] for i in 1:num_states];

### Grid of relative Pareto weights and consumption on the grid points

In [796]:
# Number of grid points on relative Pareto weight (x)
num_grid_x = 400;

# Grid points of relative Pareto weights
xmin = util_prime(maximum(inc2), σ₂) / util_prime(minimum(inc1 * (1.0 - ϕ)), σ₁);
xmax = util_prime(minimum(inc2 * (1.0 - ϕ)), σ₂) / util_prime(maximum(inc1), σ₁);
x_grid = exp.(range(log(xmin), log(xmax), length = num_grid_x));


In [798]:
# The grid points of consumption of HH 1
# Consumption is determined by state(or, aggregate income) and
# relative Pareto weights (x)
cons1 = zeros(num_states, num_grid_x)
for k in 1:num_states, l in 1:num_grid_x
    if σ₁ == σ₂
        cons1[k, l] = inc_ag[k] / (1 + x_grid[l]^(- 1.0 / σ₁))
    else
        cons1[k, l] = find_zero(
            w -> util_prime((inc_ag[k] - w), σ₂) - x_grid[l] * util_prime(w, σ₁),
            inc_mat[k][1]
        )
    end
end
cons2 = inc_ag .- cons1;

### Values under risk-sharing (full)

In [799]:
function V_full_func(
        inc1,
        P1,
        inc2,
        P2,
        num_grid_x,
        R,
        num_states,
        inc_ag,
        xmin,
        xmax,
        x_grid,
        δ,
        σ₁,
        σ₂,
        ϕ,
        util,
        util_prime,
        Uaut,
        cons1
    )
    
    # Value function iterations
    # Initial guess is expected utilities under autarky
    
    V1 = [Uaut[i][1] for i in 1:num_states] .* ones(num_grid_x)';
    V2 = [Uaut[i][2] for i in 1:num_states] .* ones(num_grid_x)';
    V1_new = zeros(num_states, num_grid_x);
    V2_new = zeros(num_states, num_grid_x);
    
    # Obtain value fundtions by value function iterations
    j = 1;
    diff = 1.0;
    while (diff > 1e-8) & (j < 500)
        V1_new = util.(cons1, σ₁) .+ δ .* (R * V1);
        V2_new = util.(inc_ag .- cons1, σ₂) .+ δ .* (R * V2);
        diff = max(maximum(abs.(V1_new - V1)), maximum(abs.(V2_new - V2)));
        V1 = V1_new;
        V2 = V2_new;
        j += 1;
    end
    
    V1_full = V1;
    V2_full = V2;
    
    return([V1_full, V2_full])
end

V_full_func (generic function with 1 method)

In [800]:
V_full = V_full_func(
  inc1, P1, inc2, P2, num_grid_x,
  R, num_states, inc_ag, xmin, xmax, x_grid,
  δ, σ₁, σ₂, ϕ,
  util, util_prime, Uaut, cons1
);
V1_full = V_full[1];
V2_full = V_full[2];

### Values under risk-sharing (limited commitment)

In [804]:
V1 = V1_full;
V2 = V2_full;
V1_new = zeros(num_states, num_grid_x);
V2_new = zeros(num_states, num_grid_x);

In [805]:
diff = 1.0
iter = 1
delta_diff = 1.0
@time while (diff > 1e-6) & (iter < 1000)

    for s_ind in 1:num_states
        x_low_ind  = findmin(abs.(util.(cons1[s_ind, :], σ₁) .+ δ .* (V1' * R[s_ind, :]) .- Uaut_1[s_ind]))[2];
        x_high_ind = findmin(abs.(util.(cons2[s_ind, :], σ₂) .+ δ .* (V2' * R[s_ind, :]) .- Uaut_2[s_ind]))[2];

        x_next_ind = collect(1:num_grid_x);
        x_next_ind[x_next_ind .<= x_low_ind] .= x_low_ind;
        x_next_ind[x_next_ind .>= x_high_ind] .= x_high_ind;

        V1_new[s_ind, :] = util.(cons1[s_ind, x_next_ind], σ₁) .+ δ .* (V1[:, x_next_ind]' * R[s_ind, :]);
        V2_new[s_ind, :] = util.(cons2[s_ind, x_next_ind], σ₂) .+ δ .* (V2[:, x_next_ind]' * R[s_ind, :]);
    end;

    diff_new = maximum(abs.(V1 .- V1_new) + abs.(V2 .- V2_new));
    delta_diff = diff - diff_new;
    diff = diff_new

    V1 = V1_new[:, :];
    V2 = V2_new[:, :];
    
    iter += 1
end

  2.017148 seconds (43.24 k allocations: 43.683 MiB, 94.00% gc time)


In [806]:
for s_ind in 1:num_states
    x_low_ind  = findmin(abs.(util.(cons1[s_ind, :], σ₁) .+ δ .* (V1' * R[s_ind, :]) .- Uaut_1[s_ind]))[2];
    x_high_ind = findmin(abs.(util.(cons2[s_ind, :], σ₂) .+ δ .* (V2' * R[s_ind, :]) .- Uaut_2[s_ind]))[2];
    print(log(x_grid[x_low_ind]), " ", log(x_grid[x_high_ind]), "\n")
end

-0.06775122817503224 0.06775122817503215
0.0364814305557865 0.6879355476234044
-0.6879355476234043 -0.036481430555786586
-0.06775122817503224 0.06775122817503215


In [547]:
# s_ind = 1;
# x_ind = 1;

# model = Model(Ipopt.Optimizer);
# set_silent(model)

# @variable(model, c[i = 1:2] >= 1e-6);
# @variable(model, x[i = 1:num_states] >= 0.0);

# register(model, :util_1, 1, x -> util(x, σ₁), x -> util_prime(x, σ₁); autodiff = true)
# register(model, :util_2, 1, x -> util(x, σ₂), x -> util_prime(x, σ₂); autodiff = true)

# register(
#     model,
#     :V1_colloc,
#     1,
#     x -> R[s_ind, :]' * [funeval(V1_collocation[i][1], basis, [x])[1][1, 1] for i in 1:num_states];
#     autodiff = true
#     );
# register(
#     model,
#     :V1_colloc,
#     1,
#     x -> R[s_ind, :]' * [funeval(V2_collocation[i][1], basis, [x])[1][1, 1] for i in 1:num_states];
#     autodiff = true
#     );

# @NLobjective(
#     model, 
#     Max, 
#     util_1(c[1]) + δ * exp_V1_colloc
# );

# @NLconstraint(
#     model, 
#     PC1,
#     util_1(c[1]) + δ * exp_V1_colloc >= Uaut_1[s_ind]
# );
# @NLconstraint(
#     model, 
#     PC2,
#     util_2(c[2]) + δ * exp_V2_colloc >= Uaut_2[s_ind]
# );

# @NLconstraint(
#     model, 
#     sum(c[i] for i in 1:2) <= inc_ag[s_ind]
# );

# @time optimize!(model)

# print(value.(c), "\n")
# @show value.(x)



  2.522635 seconds (6.66 M allocations: 386.159 MiB, 6.11% gc time, 98.58% compilation time)
[0.6733419515568383, 0.6599852697798373]
value.(x) = [1.2719914930705207, 1.2719095278593964, 1.2719095278593964, 1.2719005603048386]


4-element Vector{Float64}:
 1.2719914930705207
 1.2719095278593964
 1.2719095278593964
 1.2719005603048386

In [395]:
# s_ind = 1;
# x_ind = 1;

# basis = fundefn(:spli, num_grid_x, xmin, xmax);
# V1_collocation = [funfitxy(basis, funnode(basis)[1], V1[i, :]) for i in 1:num_states];
# V2_collocation = [funfitxy(basis, funnode(basis)[1], V2[i, :]) for i in 1:num_states];

# x_sol = zeros(num_states, num_grid_x);

# @time for s_ind in 1:num_states, x_ind in 1:num_grid_x
#     model = Model(Ipopt.Optimizer);
#     set_silent(model)

#     @variable(model, c[i = 1:2] >= 1e-10);
#     @variable(model, x >= 0.0);

#     register(model, :util_1, 1, x -> util(x, σ₁), x -> util_prime(x, σ₁); autodiff = true)
#     register(model, :util_2, 1, x -> util(x, σ₂), x -> util_prime(x, σ₂); autodiff = true)

#     register(
#         model,
#         :V1_colloc,
#         1,
#         x -> R[s_ind, :]' * collect([funeval(V1_collocation[i][1], basis, [x])[1][1,1] for i in 1:num_states]);
#         autodiff = true
#         );
#     register(
#         model,
#         :V2_colloc,
#         1,
#         x -> R[s_ind, :]' * collect([funeval(V2_collocation[i][1], basis, [x])[1][1,1] for i in 1:num_states]);
#         autodiff = true
#         );

#     @NLobjective(
#         model, 
#         Max, 
#         util_1(c[1]) + δ * V1_colloc(x)
#     );

#     @NLconstraint(
#         model, 
#         util_1(c[1]) + δ * V1_colloc(x) >= Uaut_1[s_ind]
#     );
#     @NLconstraint(
#         model, 
#         util_2(c[2]) + δ * V2_colloc(x) >= Uaut_2[s_ind]
#     );

#     @NLconstraint(
#         model, 
#         sum(c[i] for i in 1:2) <= inc_ag[s_ind]
#     );

#     optimize!(model)
    
#     print(value.(c), "\n")
#     x_sol[s_ind, x_ind] = value(x)
# end

[0.6733410768677762, 0.6599835548167918]
[0.6733410768677762, 0.6599835548167918]
[0.6733410768677762, 0.6599835548167918]
[0.6733410768677762, 0.6599835548167918]
[0.6733410768677762, 0.6599835548167918]
[0.6733410768677762, 0.6599835548167918]
[0.6733410768677762, 0.6599835548167918]
[0.6733410768677762, 0.6599835548167918]
[0.6733410768677762, 0.6599835548167918]
[0.6733410768677762, 0.6599835548167918]
[0.6733410768677762, 0.6599835548167918]
[0.6733410768677762, 0.6599835548167918]
[0.6733410768677762, 0.6599835548167918]
[0.6733410768677762, 0.6599835548167918]
[0.6733410768677762, 0.6599835548167918]
[0.6733410768677762, 0.6599835548167918]
[0.6733410768677762, 0.6599835548167918]
[0.6733410768677762, 0.6599835548167918]
[0.6733410768677762, 0.6599835548167918]
[0.6733410768677762, 0.6599835548167918]
[0.6733410768677762, 0.6599835548167918]
[0.6733410768677762, 0.6599835548167918]
[0.6733410768677762, 0.6599835548167918]
[0.6733410768677762, 0.6599835548167918]
[0.6733410768677

[1.0298794475753812, 0.9701076437865868]
[1.0298794475753812, 0.9701076437865868]
[1.0298794475753812, 0.9701076437865868]
[1.0298794475753812, 0.9701076437865868]
[1.0298794475753812, 0.9701076437865868]
[1.0298794475753812, 0.9701076437865868]
[1.0298794475753812, 0.9701076437865868]
[1.0298794475753812, 0.9701076437865868]
[1.0298794475753812, 0.9701076437865868]
[1.0298794475753812, 0.9701076437865868]
[1.0298794475753812, 0.9701076437865868]
[1.0298794475753812, 0.9701076437865868]
[1.0298794475753812, 0.9701076437865868]
[1.0298794475753812, 0.9701076437865868]
[1.0298794475753812, 0.9701076437865868]
[1.0298794475753812, 0.9701076437865868]
[1.0298794475753812, 0.9701076437865868]
[1.0298794475753812, 0.9701076437865868]
[1.0298794475753812, 0.9701076437865868]
[1.0298794475753812, 0.9701076437865868]
[1.0298794475753812, 0.9701076437865868]
[1.0298794475753812, 0.9701076437865868]
[1.0298794475753812, 0.9701076437865868]
[1.0298794475753812, 0.9701076437865868]
[1.0298794475753

[0.9956693743601714, 1.00431814104027]
[0.9956693743601714, 1.00431814104027]
[0.9956693743601714, 1.00431814104027]
[0.9956693743601714, 1.00431814104027]
[0.9956693743601714, 1.00431814104027]
[0.9956693743601714, 1.00431814104027]
[0.9956693743601714, 1.00431814104027]
[0.9956693743601714, 1.00431814104027]
[0.9956693743601714, 1.00431814104027]
[0.9956693743601714, 1.00431814104027]
[0.9956693743601714, 1.00431814104027]
[0.9956693743601714, 1.00431814104027]
[0.9956693743601714, 1.00431814104027]
[0.9956693743601714, 1.00431814104027]
[0.9956693743601714, 1.00431814104027]
[0.9956693743601714, 1.00431814104027]
[0.9956693743601714, 1.00431814104027]
[0.9956693743601714, 1.00431814104027]
[0.9956693743601714, 1.00431814104027]
[0.9956693743601714, 1.00431814104027]
[0.9956693743601714, 1.00431814104027]
[0.9956693743601714, 1.00431814104027]
[0.9956693743601714, 1.00431814104027]
[0.9956693743601714, 1.00431814104027]
[0.9956693743601714, 1.00431814104027]
[0.9956693743601714, 1.00

[1.3466824563308375, 1.3199672152672564]
[1.3466824563308375, 1.3199672152672564]
[1.3466824563308375, 1.3199672152672564]
[1.3466824563308375, 1.3199672152672564]
[1.3466824563308375, 1.3199672152672564]
[1.3466824563308375, 1.3199672152672564]
[1.3466824563308375, 1.3199672152672564]
[1.3466824563308375, 1.3199672152672564]
[1.3466824563308375, 1.3199672152672564]
[1.3466824563308375, 1.3199672152672564]
[1.3466824563308375, 1.3199672152672564]
[1.3466824563308375, 1.3199672152672564]
[1.3466824563308375, 1.3199672152672564]
[1.3466824563308375, 1.3199672152672564]
[1.3466824563308375, 1.3199672152672564]
[1.3466824563308375, 1.3199672152672564]
[1.3466824563308375, 1.3199672152672564]
[1.3466824563308375, 1.3199672152672564]
[1.3466824563308375, 1.3199672152672564]
[1.3466824563308375, 1.3199672152672564]
[1.3466824563308375, 1.3199672152672564]
[1.3466824563308375, 1.3199672152672564]
[1.3466824563308375, 1.3199672152672564]
[1.3466824563308375, 1.3199672152672564]
[1.3466824563308